Introduction: Business Problem

This project will seek the best location for the hospital. Specifically, we are interested in exploring the establishment of a new hospital for infectious diseases in Toronto, Canada. We are targeting stakeholders, especially health care administration.

There are many hospitals in Toronto, so we look for areas that are not already densely populated with hospitals. We are also interested in areas where there are no infection control hospitals in the vicinity. Also, assuming the first two conditions are met, we would like to be as close to the center of the city as possible and .

We have used the power of data science to identify some of the most promising areas based on these criteria. Generate. And clearly represent the benefits of each area so that stakeholders can ultimately choose the best location. I will.

Data 

Based on the definition of the problem, the factors that influence our decision are as follows

The number of existing hospitals in the neighborhood (any type of hospital)
The number and distance of infectious disease control hospitals in the neighborhood, if any
Distance from the city center
Using a grid that is centered around the city center and spaced at regular intervals to define the neighborhood to.

In order to extract and generate the required information, the following data sources are required

The center of the candidate region was generated algorithmically, using reverse geocoding of certain APIs to to get the approximate address of the center of those regions.
Get the number and type of hospitals in each region and their locations using the API.
The coordinates for central Toronto are based on the well-known Toronto location (Caledon) API It is obtained using geocoding.

In [2]:
import requests

def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] 
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'Toronto, Canada'
Toronto_center = get_coordinates(google_api_key, address)
print('Coordinate of {}: {}'.format(address, berlin_center))

NameError: name 'google_api_key' is not defined

In [4]:
import shapely.geometry

#!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Berlin center longitude={}, latitude={}'.format(Toronto_center[1], berlin_center[0]))
x, y = lonlat_to_xy(Toronto_center[1], Toronto_center[0])
print('Berlin center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Berlin center longitude={}, latitude={}'.format(lo, la))

ModuleNotFoundError: No module named 'shapely'

In [5]:

k = math.sqrt(3) / 2 
x_min = Toronto_center_x - 6000
x_step = 600
y_min = Toronto_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(Toronto_center_x, Toronto_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

NameError: name 'math' is not defined

In [7]:
pip install folium

     |████████████████████████████████| 93 kB 378 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
import folium

In [9]:
folium.Marker(Toronto_center, popup='Caledon').add_to(map_Toronto)
for lat, lon in zip(latitudes, longitudes):
    
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_Toronto)
    
map_Toronoto

NameError: name 'Toronto_center' is not defined

In [10]:
def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(google_api_key, Toronto_center[0], Toronto_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(Toronto_center[0], Toronto_center[1], addr))

NameError: name 'google_api_key' is not defined

In [11]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(google_api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', Canada', '')
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses: 

NameError: name 'latitudes' is not defined

In [13]:
addresses[1303:1707]

[]

In [14]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

NameError: name 'latitudes' is not defined

In [15]:
df_locations.to_pickle('./locations.pkl')   

NameError: name 'df_locations' is not defined

Methodology section

In this project, the density of hospitals in the Fatty's area, especially in areas where there are few infectious disease control hospitals, is We focus on detecting low areas. The analysis is limited to areas within 6 km from the center of the Fatty Square.

In the first step, we collected the necessary data: 6 km from the center of the Fat Fatty (Fatty Square) We collected the location and type (category) of all hospitals located within We also identified infectious disease control hospitals.

The second stage of our analysis was to calculate and study the "hospital density" across the different areas of Fatty's.

The third and final step was to focus on the most promising areas, which were established in discussions with stakeholders and We will create a cluster of locations that meet the basic requirements: we have a radius of 250 meters Considering the location where there are no more than two hospitals in a 400-meter radius, we have an infectious disease control hospital within a 400-meter radius We want places that do not have In addition to presenting a map of these places, we also want a cluster of these places (k-means (using clustering) to identify general zones/neighborhoods/addresses, and to create a final It will be a starting point for a "street level" search and a search by stakeholders for the best venue location 

Results & Analysis 

In [16]:
location_hospitals_count = [len(res) for res in location_hospitals]

df_locations['Hospitals in area'] = location_hospitals_count

print('Average number of hospitals in every area with radius=300m:', np.array(location_hospitals_count).mean())

df_locations.head(10)

NameError: name 'location_hospitals' is not defined

In [17]:
distances_to_hospital_for_infectious_diseases = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in italian_hospitals.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_hospital_for_infectious_diseases.append(min_distance)

df_locations['Distances_to_hospital_for_infectious_diseases'] = distances_to_hospital_for_infectious_diseases

NameError: name 'xs' is not defined

In [18]:
df_locations.head(10)

NameError: name 'df_locations' is not defined

In [19]:
print('Average distance to closest distances_to_hospital_for_infectious_diseases from each area center:', df_locations['Distance to distances_to_hospital_for_infectious_diseases'].mean())

NameError: name 'df_locations' is not defined

Discussion

Our analysis shows that while there are a great many hospitals in Fatty (our first area of focus was (about 2000 shops in 12x12 km around the fatty area), places near the center with low hospital density. We have found that the Therefore, we have identified the areas to the south, southeast, and east, corresponding to the southeast corner of the Fatty, Fatty, and Fatty Central Wards. We focused on it. But we focused on Fatty and Fatty, not far from residential areas, close to the center, socio-economic It is characterized by a number of locations with strong target dynamics and low hospital density.
 
After focusing on a narrow area about 5x5 km southeast of Fatty, we first looked at the dense density of candidate locations A grid (spaced 100m apart) was created; those locations are two or more houses within a 250m radius. It was filtered to locations with hospitals and locations with hospitals within 400 meters.
 
These candidate locations were then clustered to create a zone of interest that contained the most candidate locations . The addresses in the center of these zones are markers for a more detailed regional analysis based on other factors/ It was generated using reverse geocoding to use as a starting point.
 
As a result, based on the number and distance of existing hospitals, the 15 zones that contain the most potential new hospital sites (both general and infectious disease hospitals) have been determined. This, of course, means that these zones are actually the best locations for new hospitals It is not a thing. The purpose of this analysis is to show that although close to the center of Fatty's, there is a high concentration of existing hospitals (especially infectious disease control hospitals) The idea was to provide information on areas that are not. The recommended zones, therefore, will ultimately take into account the nearby competition as well as other factors, and other results where all the relevant conditions are not only met, but also have no other factors It should only be considered as a starting point for a more detailed analysis that can be produced.


Conclusion

The purpose of this project is to assist stakeholders in finding the best location for a new hospital The idea is to identify areas that are close to urban centres and have a low number of hospitals (especially hospitals for infectious diseases). By calculating the density distribution of hospitals from the data, we first identify the general boroughs that need further analysis Identify, and then identify, a larger location that meets the basic requirements for existing hospitals in the neighborhood. collections. We then clustered those locations to determine the major zones of interest (the most potential locations) as a starting point for a final investigation by stakeholders. Zone center addresses have been created.

The need for each location (distance to other hospitals and ease of access), noise levels/to major roads. Additional factors such as proximity, property availability, price, and social and economic dynamics of each region. Based on the specific characteristics of the neighborhoods and locations within each recommended zone, while taking into account the best Stakeholders make the final decision on hospital location.